In [ ]:
import numpy as np
import pandas as pd


In [ ]:
!pip3 install transformers 
!pip3 install torch senetencepiece

     |████████████████████████████████| 3.3 MB 4.1 MB/s 
     |████████████████████████████████| 61 kB 507 kB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement senetencepiece (from versions: none)
ERROR: No matching distribution found for senetencepiece


In [ ]:
from transformers import pipeline

model = pipeline('summarization', max_length=25, min_length=15)

text = """Italy's antitrust authority has imposed a fine of 1.13 billion euros ($1.28 billion) on Amazon for alleged abuse of market dominance. Amazon leveraged its dominant position "to favour the adoption of its own logistics service - Fulfilment by Amazon (FBA) - by sellers active on Amazon", the watchdog said. The authority said it will also impose corrective steps on Amazon."""

summary = model(text)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
print(summary[0]['summary_text'])

 Italy's antitrust authority has imposed a fine of 1.13 billion euros ($1.28 billion) on Amazon


<h2>Creating Training Data</h2>

In [ ]:
text_array = []
summary_array = []

text_data = pd.read_excel('/content/cricket_text.xlsx')
text_data.head()

text_array = text_data.to_numpy()

for i in range(0,len(text_array)):
  summary_array.append(model(text_array[i][0])[0]['summary_text'])

#training_data = pd.DataFrame({'text':text_array,'summary':summary_array})

In [ ]:
text_array = text_array.ravel()
print(text_array[0])
print(summary_array[0])

The BCCI today announced Team India's 18-member squad for the upcoming Test series against South Africa. Rohit Sharma has been named vice-captain for the series. Hanuma Vihari, who wasn't included in the squad for the Test series against New Zealand, has returned to the squad. The squad also includes Jasprit Bumrah, Shardul Thakur and Mohammad Shami.
 Rohit Sharma has been named vice-captain for the series against South Africa . Hanuma Vihari


In [ ]:
print(len(text_array))
print(len(summary_array))

108
108


In [ ]:
training_data = pd.DataFrame({'text':text_array,'summary':summary_array})
training_data.head()
training_data.to_csv('outputs/CricketSummaryData.csv',index=False)


<h2>Importing T5 Model and train_test_split</h2>


In [ ]:
!pip install simplet5

     |████████████████████████████████| 1.2 MB 4.2 MB/s 
     |████████████████████████████████| 2.8 MB 33.6 MB/s 
     |████████████████████████████████| 919 kB 42.1 MB/s 
     |████████████████████████████████| 132 kB 50.4 MB/s 
     |████████████████████████████████| 332 kB 46.3 MB/s 
     |████████████████████████████████| 829 kB 43.3 MB/s 
     |████████████████████████████████| 1.1 MB 33.1 MB/s 
     |████████████████████████████████| 160 kB 51.1 MB/s 
     |████████████████████████████████| 192 kB 52.3 MB/s 
     |████████████████████████████████| 271 kB 51.2 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.3-py3-none-any.whl size=6831 sha256=4c2506c61b30149a3df309e627a72b9aad039b13f2f190d6ab2f757481f36b55
  Stored in directory: /root/.cache/pip/wheels/fb/87/9d/190f8091eed71a654458ca3a285743ed9ffeb55c2ffa4be68f
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=122c7832180d37801440e3f196f93800d5124c8a15b8ee67b2c055febd6a0834
  Stored 

In [ ]:
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5

Global seed set to 42


Renaming data columns to source_text and target_text

In [ ]:
df = pd.read_csv('/content/outputs/CricketSummaryData.csv')
df = df.rename(columns={'text':'source_text','summary':'target_text'})
df.head()

,source_text,target_text
0,The BCCI today announced Team India's 18-membe...,Rohit Sharma has been named vice-captain for ...
1,Pat Cummins took a five-wicket haul on his cap...,Pat Cummins took a five-wicket haul on his ca...
2,Reacting to Rohit Sharma replacing Virat Kohli...,Harsha Bhogle says Virat Kohli will feel a se...
3,Ex-Australia leg-spinner Shane Warne repeatedl...,Shane Warne repeatedly said Mitchell Starc's ...
4,Ex-England captain Nasser Hussain has said he ...,Ex-England captain Nasser Hussain says he wou...


Adding prefix to uniquely identify task to be performed

In [ ]:
df['source_text'] = "summarize: " + df['source_text']
df.head()

,source_text,target_text
0,summarize: The BCCI today announced Team India...,Rohit Sharma has been named vice-captain for ...
1,summarize: Pat Cummins took a five-wicket haul...,Pat Cummins took a five-wicket haul on his ca...
2,summarize: Reacting to Rohit Sharma replacing ...,Harsha Bhogle says Virat Kohli will feel a se...
3,summarize: Ex-Australia leg-spinner Shane Warn...,Shane Warne repeatedly said Mitchell Starc's ...
4,summarize: Ex-England captain Nasser Hussain h...,Ex-England captain Nasser Hussain says he wou...


Splitting data into training and test data

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3)
print(train_df.shape)
print(test_df.shape)

(75, 2)
(33, 2)


Instantiating T5 model


In [ ]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Training the model

In [ ]:
model.train(
    train_df = train_df[:5000],
    eval_df = test_df[:100],
    source_max_token_len = 120,
    target_max_token_len = 50,
    batch_size = 8,
    max_epochs = 5,
    use_gpu = True
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

viewing pre-trained models

In [ ]:
! ( cd outputs; ls )

#select the model with min loss

CricketSummaryData.csv		    simplet5-epoch-2-train-loss-0.3233
simplet5-epoch-0-train-loss-0.8921  simplet5-epoch-3-train-loss-0.2408
simplet5-epoch-1-train-loss-0.5121  simplet5-epoch-4-train-loss-0.1632


Load the model


In [ ]:
model.load_model("t5","outputs/simplet5-epoch-4-train-loss-0.1632",use_gpu=True)

Summarizing text


In [ ]:
text_to_summarize = """India off-spinner Ravichandran Ashwin tried to review umpire's decision after getting bowled off New Zealand's Ajaz Patel in second Test today. Ashwin got clean bowled after which the umpire raised his finger. Ashwin was unaware he got bowled and seemingly thought he got caught behind and signalled for review. However, the umpire used video referral to confirm he was bowled."""
model.predict(text_to_summarize)

["Ravichandran Ashwin tried to review umpire's decision after getting bowled off Ajaz Patel in second Test today."]